In [1]:
pip install sounddevice

   ---------------------------------------- 0.0/200.1 kB ? eta -:--:--
   ----- --------------------------------- 30.7/200.1 kB 660.6 kB/s eta 0:00:01
   ---------------------------------------- 200.1/200.1 kB 3.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [16]:
import numpy as np
import sounddevice as sd

def lfsr(seed, taps, length):
    lfsr = seed
    seq = []
    for _ in range(length):
        # XOR the bits at the specified tap positions
        bit = -1
        for tap in taps:
            if bit == -1:
                bit = lfsr[tap]
            else:
                bit ^= lfsr[tap]
        seq.append(lfsr[-1])
        lfsr = np.roll(lfsr, 1)
        lfsr[0] = bit
    return np.array(seq)

def generate_square_wave(lfsr_sequence, sample_rate, frequency):
    t = np.linspace(0, len(lfsr_sequence) / sample_rate, len(lfsr_sequence), endpoint=False)
    square_wave = np.sign(np.sin(2 * np.pi * frequency * t))
    # Modulate square wave with LFSR sequence
    waveform = square_wave * (2 * lfsr_sequence - 1)  # Convert LFSR (0,1) to (-1,1)
    return waveform

def main():
    taps_more = [[2,3],[6,3,7],[4,8,2,1,0,6]]
    # LFSR configuration
    seed = np.array([1, 0, 0, 1, 0, 0, 0, 1, 1])  # Initial state of the LFSR (9 bits)
    length = 44100  # Length of the LFSR sequence

    for taps in taps_more:
        # Generate LFSR sequence
        lfsr_seq = lfsr(seed, taps, length)

        # Audio configuration
        sample_rate = 44100  # Sampling rate in Hz
        frequency = 440  # Frequency of the square wave (A4 note)

        # Generate the tone based on the LFSR sequence
        tone = generate_square_wave(lfsr_seq, sample_rate, frequency)

        # Play the generated tone
        sd.play(tone, sample_rate)
        sd.wait()  # Wait until the sound has finished playing

if __name__ == "__main__":
    main()
